In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]= os.getenv("GROQ_API_KEY")

#Summarization Middleware

In [2]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage,SystemMessage


agent = create_agent(
  model = "groq:llama-3.1-8b-instant",
  checkpointer=InMemorySaver(),
  middleware=[
    SummarizationMiddleware(
      model = "groq:llama-3.1-8b-instant",
      trigger = ("messages",10),
      keep =("messages",4)
    )
    ]
)

e:\ONEXT\AI Research\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config = {"configurable":{"thread_id":"test_1"}}

In [4]:
questions = [
  "what is 2+2",
  "what is 10*5",
  "what is 10/2",
  "what is 10-2",
  "what is 4*4",
  "what is a 10/10"
]

for q in questions:
  response = agent.invoke({"messages":[HumanMessage(content=q)]},config) 
  print(f"Messages:{response}")
  print(f"Messages:{len(response['messages'])}")

Messages:{'messages': [HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}, id='b6643973-d8ed-4235-91e9-63d4363fcf84'), AIMessage(content='2 + 2 = 4', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 41, 'total_tokens': 49, 'completion_time': 0.006246706, 'completion_tokens_details': None, 'prompt_time': 0.003370843, 'prompt_tokens_details': None, 'queue_time': 0.060283516, 'total_time': 0.009617549}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bc478-d0f5-77a1-8fc4-633382a9c61a-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 41, 'output_tokens': 8, 'total_tokens': 49})]}
Messages:2
Messages:{'messages': [HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}, id='b6643973-d8ed-4235-91e9-63d4363fcf84'), AIMessage(conte

### Token size

In [9]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage,SystemMessage
from langchain_core.tools import tool

@tool
def search_hotels(city:str)->str:
  """Search for hotels in a specific city based on location."""
  return f"""Hotels in {city}":
  1. Hotel A - 5 stars, 350$/night, spa, pool, gym
  2. Hotel B - 4 stars, 250$/night, pool, gym
  3. Hotel C - 3 stars, 150$/night, free wifi
  """

agent = create_agent(
  model = "groq:llama-3.1-8b-instant",
  tools= [search_hotels],
  checkpointer=InMemorySaver(),
  middleware=[
    SummarizationMiddleware(
      model = "groq:llama-3.1-8b-instant",
      trigger = ("tokens",550),
      keep =("tokens",200)
    )
    ]
)

config = {"configurable":{"thread_id":"test_1"}}

#token counter
def count_tokens(messages):
  total = sum(len(str(m.content)) for m in messages)
  return total

In [10]:
cities = ["New York", "Los Angeles", "Chicago", "Houston", "Phoenix"]

for city in cities:
  response = agent.invoke(
    {"messages":[HumanMessage(content=f"Find hotels in {city}")]},
    config = config
  )
  token = count_tokens(response["messages"])
  print(f"{city}:~{token} tokens,{len(response['messages'])} messages")
  print(f"{response['messages']}")

New York:~334 tokens,4 messages
[HumanMessage(content='Find hotels in New York', additional_kwargs={}, response_metadata={}, id='013d918d-7a69-4fb3-92db-9ac0e53aa4cf'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '66ppn5a6k', 'function': {'arguments': '{"city":"New York"}', 'name': 'search_hotels'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 227, 'total_tokens': 243, 'completion_time': 0.073592659, 'completion_tokens_details': None, 'prompt_time': 0.017339891, 'prompt_tokens_details': None, 'queue_time': 0.104304329, 'total_time': 0.09093255}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bc479-655e-7a10-9b76-a7e4a2567351-0', tool_calls=[{'name': 'search_hotels', 'args': {'city': 'New York'}, 'id': '66ppn5a6k', 'type': 'tool_call'}], invalid_tool_calls=[], usage_

### Human in the loop middelware

In [24]:
from langchain.agents import  create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

def read_email_tool(email_id:str)->str:
    """Mock function to read an email by its ID."""
    return f"Emanil content for ID:{email_id}"

def send_email_tool(recipient:str, subject:str, body:str)->str:
  """Mock function to send an email"""
  return f"Email sent to {recipient} with subject '{subject}'"

In [23]:
agent = create_agent(
  model = "groq:llama-3.1-8b-instant",
  tools= [read_email_tool,send_email_tool],
  checkpointer=InMemorySaver(),
  middleware=[
      HumanInTheLoopMiddleware(
        interrupt_on={
          "send_email_tool":{
            "allowed_decisions":["approve","edit","reject"]
            },
            "read_email_tool":False,
          }
        )
    ]
)

In [ ]:
config={"configurable":{"thread_id":"test_approve"}}

result = agent.invoke(
  {"messages":[HumanMessage(content ="Send email to giang@test.com with subject 'Xin chào' and body 'Bạn khỏe không'")]},
  config=config
)

In [27]:
result

{'messages': [HumanMessage(content="Send email to giang@test.com with subject 'Xin chào' and body 'Bạn khỏe không'", additional_kwargs={}, response_metadata={}, id='868084ac-1976-4783-b254-5c57b842a454'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'wh1qwbhna', 'function': {'arguments': '{"body":"Đàng khôe không","recipient":"giang@test.com","subject":"Xin cháo"}', 'name': 'send_email_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 313, 'total_tokens': 370, 'completion_time': 0.107257268, 'completion_tokens_details': None, 'prompt_time': 0.019526849, 'prompt_tokens_details': None, 'queue_time': 0.050976585, 'total_time': 0.126784117}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bc47c-5734-7ee0-89e4-493506fcbeab-0', tool_calls=[{'name': 'send_email_tool', '

In [28]:
from langgraph.types import Command

if "__interrupt__" in result:
  print("Paused! Approving ...")

  result = agent.invoke(
    Command(
      resume={
        "decisions":[
          {"type":"approve"}
        ]
      }
    ),
    config=config
  )

  print(f"Result: {result['messages'][-1].content}")

Paused! Approving ...
Result: 


In [29]:
result

{'messages': [HumanMessage(content="Send email to giang@test.com with subject 'Xin chào' and body 'Bạn khỏe không'", additional_kwargs={}, response_metadata={}, id='868084ac-1976-4783-b254-5c57b842a454'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'wh1qwbhna', 'function': {'arguments': '{"body":"Đàng khôe không","recipient":"giang@test.com","subject":"Xin cháo"}', 'name': 'send_email_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 313, 'total_tokens': 370, 'completion_time': 0.107257268, 'completion_tokens_details': None, 'prompt_time': 0.019526849, 'prompt_tokens_details': None, 'queue_time': 0.050976585, 'total_time': 0.126784117}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bc47c-5734-7ee0-89e4-493506fcbeab-0', tool_calls=[{'name': 'send_email_tool', '

### Reject

In [30]:
from langchain.agents import  create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

def read_email_tool(email_id:str)->str:
    """Mock function to read an email by its ID."""
    return f"Emanil content for ID:{email_id}"

def send_email_tool(recipient:str, subject:str, body:str)->str:
  """Mock function to send an email"""
  return f"Email sent to {recipient} with subject '{subject}'"


  agent = create_agent(
  model = "groq:llama-3.1-8b-instant",
  tools= [read_email_tool,send_email_tool],
  checkpointer=InMemorySaver(),
  middleware=[
      HumanInTheLoopMiddleware(
        interrupt_on={
          "send_email_tool":{
            "allowed_decisions":["approve","edit","reject"]
            },
            "read_email_tool":False,
          }
        )
    ]
)

In [31]:
config={"configurable":{"thread_id":"test_approve"}}

result = agent.invoke(
  {"messages":[HumanMessage(content ="Send email to giang@test.com with subject 'Xin chào' and body 'Bạn khỏe không'")]},
  config=config
)

In [32]:
from langgraph.types import Command

if "__interrupt__" in result:
  print("Paused! Approving ...")

  result = agent.invoke(
    Command(
      resume={
        "decisions":[
          {"type":"reject"}
        ]
      }
    ),
    config=config
  )

  print(f"Result: {result['messages'][-1].content}")

Paused! Approving ...
Result: 


In [33]:
result

{'messages': [HumanMessage(content="Send email to giang@test.com with subject 'Xin chào' and body 'Bạn khỏe không'", additional_kwargs={}, response_metadata={}, id='868084ac-1976-4783-b254-5c57b842a454'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'wh1qwbhna', 'function': {'arguments': '{"body":"Đàng khôe không","recipient":"giang@test.com","subject":"Xin cháo"}', 'name': 'send_email_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 313, 'total_tokens': 370, 'completion_time': 0.107257268, 'completion_tokens_details': None, 'prompt_time': 0.019526849, 'prompt_tokens_details': None, 'queue_time': 0.050976585, 'total_time': 0.126784117}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bc47c-5734-7ee0-89e4-493506fcbeab-0', tool_calls=[{'name': 'send_email_tool', '

### Edit

In [34]:
from langchain.agents import  create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

def read_email_tool(email_id:str)->str:
    """Mock function to read an email by its ID."""
    return f"Emanil content for ID:{email_id}"

def send_email_tool(recipient:str, subject:str, body:str)->str:
  """Mock function to send an email"""
  return f"Email sent to {recipient} with subject '{subject}'"

In [35]:
agent = create_agent(
  model = "groq:llama-3.1-8b-instant",
  tools= [read_email_tool,send_email_tool],
  checkpointer=InMemorySaver(),
  middleware=[
      HumanInTheLoopMiddleware(
        interrupt_on={
          "send_email_tool":{
            "allowed_decisions":["approve","edit","reject"]
            },
            "read_email_tool":False,
          }
        )
    ]
)

In [36]:
config={"configurable":{"thread_id":"test_approve"}}

result = agent.invoke(
  {"messages":[HumanMessage(content ="Send email to giang@test.com with subject 'Xin chào' and body 'Bạn khỏe không'")]},
  config=config
)

In [37]:
result

{'messages': [HumanMessage(content="Send email to giang@test.com with subject 'Xin chào' and body 'Bạn khỏe không'", additional_kwargs={}, response_metadata={}, id='1feffc17-7cf8-4376-901b-8e34aabe0660'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '52jdvhzaq', 'function': {'arguments': '{"body":"Bạn khỏe không","recipient":"giang@test.com","subject":"Xin cháo"}', 'name': 'send_email_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 313, 'total_tokens': 353, 'completion_time': 0.047039626, 'completion_tokens_details': None, 'prompt_time': 0.022162316, 'prompt_tokens_details': None, 'queue_time': 0.050163434, 'total_time': 0.069201942}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bc47f-cbf3-7c90-ae37-6ac4376b90d1-0', tool_calls=[{'name': 'send_email_tool', 'a

In [39]:
from langgraph.types import Command

if "__interrupt__" in result:
  print("Paused! Eddting ...")

  result = agent.invoke(
    Command(
      resume={
        "decisions":[
          {
            "type":"edit",
            "edited_action":{
              "name":"send_email_tool",
              "args":{
                "recipient":"corect@email.com",
                "subject":"Corrected subject",
                "body":"This was edited by human before sending"
              }
            }
          }
        ]
      }
    ),
    config=config
  )

  print(f"Result: {result['messages'][-1].content}")

Paused! Eddting ...
Result: 


In [40]:
result

{'messages': [HumanMessage(content="Send email to giang@test.com with subject 'Xin chào' and body 'Bạn khỏe không'", additional_kwargs={}, response_metadata={}, id='1feffc17-7cf8-4376-901b-8e34aabe0660'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '52jdvhzaq', 'function': {'arguments': '{"body":"Bạn khỏe không","recipient":"giang@test.com","subject":"Xin cháo"}', 'name': 'send_email_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 313, 'total_tokens': 353, 'completion_time': 0.047039626, 'completion_tokens_details': None, 'prompt_time': 0.022162316, 'prompt_tokens_details': None, 'queue_time': 0.050163434, 'total_time': 0.069201942}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bc47f-cbf3-7c90-ae37-6ac4376b90d1-0', tool_calls=[{'name': 'send_email_tool', 'a